In [1]:
import pandas as pd
import numpy as np

# Combining and Merging Datasets

In [2]:
df1 = pd.DataFrame({
    'key': 'b b a c a a b'.split(),
    'data1': range(7)
})

In [3]:
df2 = pd.DataFrame({
    'key': 'a b d'.split(),
    'data2': range(3)
})

In [4]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [5]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [6]:
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


If the column names are different?

In [7]:
df3 = pd.DataFrame({
    'lkey': 'b b a c a a b'.split(),
    'data1': range(7)
})

In [8]:
df4 = pd.DataFrame({
    'rkey': 'a b d'.split(),
    'data2': range(3)
})

In [9]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


Left join? Inner join? Well the default is inner. But we can specify.

In [10]:
pd.merge(df1, df2, on='key', how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


## Many to many

In [11]:
df1 = pd.DataFrame({
    'key': 'b b a c a b'.split(),
    'data1': range(6)
})
df2 = pd.DataFrame({
    'key': 'a b a b d'.split(),
    'data2': range(5)
})

In [12]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [13]:
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [14]:
pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


Merge with multiple keys

In [15]:
left = pd.DataFrame({
    'key1': 'foo foo bar'.split(),
    'key2': 'one two one'.split(),
    'lval': [1, 2, 3]
})

In [16]:
right = pd.DataFrame({
    'key1': 'foo foo bar bar'.split(),
    'key2': 'one one one two'.split(),
    'rval': [4, 5, 6, 7]
})

In [17]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [18]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


## Merging on Index

In [19]:
left1 = pd.DataFrame({
    'key': 'a b a a b c'.split(),
    'value': range(6)
})

In [20]:
right1 = pd.DataFrame({
    'group_val': [3.5, 7]
}, index=['a', 'b'])

In [21]:
right1

,group_val
a,3.5
b,7.0


In [22]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


I like that. This is so elegant.

How about an outer join?

In [23]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


## Hierarchical merges

In [24]:
lefth = pd.DataFrame({
    'key1': 'Ohio Ohio Ohio Nevada Nevada'.split(),
    'key2': [2000, 2001, 2002, 2001, 2002],
    'data': np.arange(5.)
})

In [25]:
righth = pd.DataFrame(
    np.arange(12).reshape((6,2)),
    index=[
        'Nevada Nevada Ohio Ohio Ohio Ohio'.split(),
        [2001, 2000, 2000, 2000, 2001, 2002]
    ],
    columns=['event1', 'event2']
)

In [26]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [27]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [28]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


Index on both sides

In [29]:
left2 = pd.DataFrame(
    [
        [1., 2.], [3., 4.], [5., 6.]
    ],
    index=['a', 'c', 'e'],
    columns=['Ohio', 'Nevada']
)

In [30]:
right2 = pd.DataFrame(
    [
        [7., 8.], [9., 10.], [11., 12.], [13., 14.]
    ],
    index=['b', 'c', 'd', 'e'],
    columns=['Missouri', 'Alabama']
)

In [31]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [32]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [33]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


Concatenate things where the index doesn't contain relevant data.

In [35]:
df1 = pd.DataFrame(np.random.randn(3,4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2,3), columns=['b', 'd', 'a'])

In [38]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.721199,-1.219320,-1.706971,2.317255
1,0.338639,-0.308023,0.773061,-0.420272
2,1.586569,-1.629009,-1.222172,-1.748692
3,0.765197,-0.003967,NaN,0.708033
4,2.088525,-0.061343,NaN,-1.250509


In [39]:
pd.concat([df1, df2])

,a,b,c,d
0,-0.721199,-1.219320,-1.706971,2.317255
1,0.338639,-0.308023,0.773061,-0.420272
2,1.586569,-1.629009,-1.222172,-1.748692
0,0.765197,-0.003967,NaN,0.708033
1,2.088525,-0.061343,NaN,-1.250509


What if you have duplicate data?

In [40]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series([0, np.nan, 2., np.nan, np.nan, 5.], index=['a', 'b', 'c', 'd', 'e', 'f'])

In [43]:
np.where(pd.isnull(a), b, a)

array([ 0. ,  2.5,  0. ,  3.5,  4.5,  5. ])

In [44]:
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

Ok, now the same thing with `DataFrame`s

In [45]:
df1 = pd.DataFrame({
    'a': [1., np.nan, 5., np.nan],
    'b': [np.nan, 2., np.nan, 6.],
    'c': range(2, 18, 4)
})

In [46]:
df2 = pd.DataFrame({
    'a': [5., 4., np.nan, 3., 7.],
    'b': [np.nan, 3., 4., 6., 8.]
})

In [49]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN
